In [1]:
import torch
import pandas as pd
import sklearn
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
from torch.utils.data import Dataset,DataLoader
from utils.nn_utils import *
from models.ViT import ViT_LRP_nan_excluded
from models.LSTM import Recurrent_Classifier

X_datapaths = ['./preprocessed/prepared/nan/L2Y1.pkl','./preprocessed/prepared/nan/L2Y2.pkl','./preprocessed/prepared/nan/L2Y3.pkl','./preprocessed/prepared/nan/L2Y4.pkl','./preprocessed/prepared/nan/L2Y5.pkl','./preprocessed/prepared/nan/L2Y6.pkl',]
label_datapath = './preprocessed/prepared/nan/label.pkl'
#X_datapaths = ['./preprocessed/prepared/fill/L2Y1.pkl','./preprocessed/prepared/fill/L2Y2.pkl','./preprocessed/prepared/fill/L2Y3.pkl','./preprocessed/prepared/fill/L2Y4.pkl','./preprocessed/prepared/fill/L2Y5.pkl','./preprocessed/prepared/fill/L2Y6.pkl',]
#label_datapath = './preprocessed/prepared/fill/label.pkl'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# read pickle
input_datas = [] # list of each input pandas dataframe
for datapath in X_datapaths:
    temp = pd.read_pickle(datapath)
    temp = temp.reset_index()
    input_datas.append(temp)

label_data = pd.read_pickle(label_datapath)
label_data = label_data.reset_index()



seq_len = len(input_datas)

label_data = label_data - 1


CLS2IDX = {
    0 : '1등급',
    1 : '2등급',
    2 : '3등급',
    3 : '4등급',
    4 : '5등급',
    5 : '6등급',
    6 : '7등급',
    7 : '8등급',
    8 : '9등급'
}
is_regression = False
X_trains, X_tests, y_train, y_test = make_splited_data(input_datas,label_data,is_regression=is_regression)

train_dataset = KELSDataSet(X_trains,y_train,is_regression=is_regression)
test_dataset = KELSDataSet(X_tests,y_test,is_regression=is_regression)


batch_size = 32
hidden_features = 100
embbed_dim = 72



train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
split_list = train_dataset.split_list
#embedding_networks : 년차별로 맞는 mlp 리스트. 리스트 내용물에 따라 인풋 채널 개수 다름.

sample_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
assert batch_size
(sample,label) = next(iter(sample_loader))
sample = sample.to(device)
sample_datas = batch_to_splited_datas(sample,split_list)


#embbeding_networks = make_embbeding_networks(sample_datas,batch_size=batch_size,hidden_features = hidden_features,out_features=embbed_dim)
#embbeding network 사용법 : 
#1. train_loader에서 배치를 받는다. 이 배치는 (batch, 학생수, features)인데, features는 모든 년차별 feature가 concat된 상태.
#2. 이 배치를 batch_to_splited_datas에 넣는다. datas = batch_to_splited_datas(배치,split_list)
# 그러면 datas에 년차별 feature가 나누어진 리스트가 생기게 된다. 리스트 길이는 년차 길이고, 내용물은 행렬
# 3. 이 datas를 batch_to_embbedings 에 넣는다. 그럼 각 행렬이 embbeding_networks에 든 mlp를 통과하여 같은 크기의 행렬들을 리턴한다. 
# emb_batch_list= batch_to_embbedings(datas,embbeding_networks)
#4. emb_batch_list의 내용물이 바로 contrastive loss에 들어갈 벡터들이 된다. 

#5. lstm에 넣으려면 얘들을 seq_len 방향으로 쌓아야 한다. 
#emb_batched_seq = torch.stack(emb_batch_list).transpose(0,1) 으로 쌓고 lstm에 넣으면 완성..?

In [2]:
def accuracy_roughly(y_pred, y_label):
    #TODO add as loss
    if len(y_pred) != len(y_label):
        print("not available, fit size first")
        return
    cnt = 0
    correct = 0
    for pred, label in zip(y_pred, y_label):
        cnt += 1
        if abs(pred-label) <= 1:
            correct += 1
    return correct / cnt

In [30]:
def train_net(model,train_loader,test_loader,optimizer_cls = optim.AdamW, criterion = nn.CrossEntropyLoss(),
n_iter=10,device='cpu',lr = 0.001,weight_decay = 0.01,mode = None):
        
        train_losses = []
        train_acc = []
        val_accs = []
        positive_accs = []
        #optimizer = optimizer_cls(model.parameters(),lr=lr,weight_decay=weight_decay)
        optimizer = optimizer_cls(model.parameters(),lr=lr)
        #scheduler = optim.lr_scheduler.MultiStepLR(optimizer,milestones=[25,40,60,80], gamma=0.5,last_epoch=-1)
        

        for epoch in range(n_iter):
                running_loss = 0.0
                model.train()
                n = 0
                n_acc = 0
                ys = []
                ypreds = []
                for i,(xx,(label_E,label_K,label_M)) in tqdm(enumerate(train_loader)):

                
                
                        xx = xx.to(device)
                        if mode == 'E':
                                yy = label_E
                        elif mode == 'K':
                                yy = label_K
                        elif mode == 'M':
                                yy = label_M
                        else:
                                assert True
                        
                        yy = yy.to(device)
                        

                        
                        
                
                        optimizer.zero_grad()
                        outputs = model(xx)
                        _,y_pred = outputs.max(1)

                        # Calculate Loss: softmax --> cross entropy loss
                        #loss = criterion(outputs, yy)
                        loss1 = criterion(outputs,yy)



                        yy_idx1 = (yy == 8)
                        yy_idx2 = (yy == 0)

                        # outputs2 = outputs.clone()
                        # outputs2[yy_idx1] = 0
                        # outputs3 = outputs.clone()
                        # outputs3[yy_idx2] = 0


                        loss2 = criterion(outputs,(yy+1).clamp(max=8))
                        loss3 = criterion(outputs,(yy-1).clamp(min=0))
                        #loss2[yy_idx1] = 0
                        #loss3[yy_idx2] = 0
                        loss = loss1 + loss2 + loss3

                        # Getting gradients w.r.t. parameters
                        loss.backward()

                        # Updating parameters
                        optimizer.step()
                        ys.append(yy)
                        ypreds.append(y_pred)
                        
                        
                        i += 1
                        n += len(xx)
                        _, y_pred = outputs.max(1)
                        n_acc += (yy == y_pred).float().sum().item()
                #scheduler.step()
                train_losses.append(running_loss/i)
                train_acc.append(n_acc/n)
                ys = torch.cat(ys)
                ypreds = torch.cat(ypreds)
                train_positive_acc = accuracy_roughly(ypreds,ys)
                acc, positive_acc = eval_net(model,test_loader,device,mode = mode)
                val_accs.append(acc)
                positive_accs.append(positive_acc)

                print(f'epoch : {epoch},train_positive_acc : {train_positive_acc} train_acc : {train_acc[-1]}, acc : {val_accs[-1]}. positive_acc : {positive_accs[-1]}',flush = True)

In [31]:
def eval_net(model,data_loader,device,mode=None):
    model.eval()
    ys = []
    ypreds = []
    for xx,(label_E,label_K,label_M) in data_loader:

                
                
        xx = xx.to(device)
        if mode == 'E':
            y = label_E
        elif mode == 'K':
            y = label_K
        elif mode == 'M':
            y = label_M
        else:
            assert True
        
        y = y.to(device)

        with torch.no_grad():
                score = model(xx)
                _,y_pred = score.max(1)
        ys.append(y)
        ypreds.append(y_pred)

    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    positive_acc = accuracy_roughly(ypreds,ys)
    acc= (ys == ypreds).float().sum() / len(ys)

    # print(sklearn.metrics.confusion_matrix(ys.numpy(),ypreds.numpy()))


    # print(sklearn.metrics.classification_report(ys.numpy(),ypreds.numpy()))
    

    return acc, positive_acc
    #return acc.item()

In [5]:
model_LSTM = Recurrent_Classifier.RecurrentClassifier(sample_datas,split_list,embedding_dim=32,hidden_dim=64,output_size=9)
model_LSTM = model_LSTM.to(device)
train_net(model_LSTM,train_loader,test_loader,n_iter=100,device=device,mode='E',lr=0.0001,optimizer_cls = optim.AdamW)

131it [00:02, 53.31it/s]


epoch : 0, train_acc : 0.129171668667467, acc : 0.19961611926555634. positive_acc : 0.40978886756238003


131it [00:02, 54.73it/s]


epoch : 1, train_acc : 0.18319327731092436, acc : 0.19961611926555634. positive_acc : 0.40978886756238003


131it [00:02, 56.14it/s]


...

epoch : 97, train_acc : 0.3915966386554622, acc : 0.3714011311531067. positive_acc : 0.7130518234165067


131it [00:02, 56.62it/s]


epoch : 98, train_acc : 0.39063625450180073, acc : 0.37236082553863525. positive_acc : 0.7072936660268714


131it [00:02, 56.15it/s]


epoch : 99, train_acc : 0.39327731092436974, acc : 0.362763911485672. positive_acc : 0.7063339731285988


In [24]:
model_E = ViT_LRP_nan_excluded.VisionTransformer(sample_datas,split_list,seq_len=6, num_classes=9, embed_dim=32*3, depth=6,
                 num_heads=8, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.2, attn_drop_rate=0.2)
model_E = model_E.to(device)

train_net(model_E,train_loader,test_loader,n_iter=100,device=device,mode='E',lr=0.0001,optimizer_cls = optim.AdamW)


131it [00:06, 19.72it/s]


epoch : 0, train_acc : 0.22593037214885955, acc : 0.27639153599739075. positive_acc : 0.5777351247600768


131it [00:07, 17.21it/s]


epoch : 1, train_acc : 0.3085234093637455, acc : 0.3272552788257599. positive_acc : 0.6525911708253359


131it [00:09, 13.11it/s]


...

epoch : 16, train_acc : 0.3971188475390156, acc : 0.3522072732448578. positive_acc : 0.6794625719769674


131it [00:07, 18.12it/s]


epoch : 17, train_acc : 0.39399759903961584, acc : 0.3522072732448578. positive_acc : 0.6948176583493282


76it [00:04, 18.82it/s]


KeyboardInterrupt: 

In [32]:
model_K = ViT_LRP_nan_excluded.VisionTransformer(sample_datas,split_list,seq_len=6, num_classes=9, embed_dim=16*3, depth=8,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.2, attn_drop_rate=0.2)
model_K = model_K.to(device)

train_net(model_K,train_loader,test_loader,n_iter=100,device=device,mode='K',lr=0.0001,optimizer_cls = optim.AdamW)


131it [00:06, 19.85it/s]


epoch : 0,train_positive_acc : 0.6014405762304922 train_acc : 0.1735894357743097, acc : 0.19577734172344208. positive_acc : 0.6861804222648752


131it [00:06, 19.93it/s]


epoch : 1,train_positive_acc : 0.7258103241296519 train_acc : 0.22713085234093638, acc : 0.2476007640361786. positive_acc : 0.7687140115163148


131it [00:06, 19.77it/s]


...

epoch : 97,train_positive_acc : 0.9147659063625451 train_acc : 0.42809123649459785, acc : 0.2715930938720703. positive_acc : 0.718809980806142


131it [00:06, 19.33it/s]


epoch : 98,train_positive_acc : 0.9133253301320529 train_acc : 0.42953181272509006, acc : 0.2831093966960907. positive_acc : 0.6948176583493282


131it [00:06, 19.64it/s]


epoch : 99,train_positive_acc : 0.9162064825930372 train_acc : 0.44129651860744296, acc : 0.2667946219444275. positive_acc : 0.708253358925144


In [ ]:
model_K = ViT_LRP_nan_excluded.VisionTransformer(sample_datas,split_list,seq_len=6, num_classes=9, embed_dim=16*3, depth=8,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.2, attn_drop_rate=0.2)
model_K = model_K.to(device)

train_net(model_K,train_loader,test_loader,n_iter=100,device=device,mode='K',lr=0.0001,optimizer_cls = optim.AdamW)


131it [00:06, 20.15it/s]


epoch : 0,train_positive_acc : 0.5707082833133253 train_acc : 0.16806722689075632, acc : 0.23128598928451538. positive_acc : 0.7428023032629558


131it [00:06, 20.24it/s]


epoch : 1,train_positive_acc : 0.7205282112845138 train_acc : 0.22737094837935173, acc : 0.2418425977230072. positive_acc : 0.7418426103646834


131it [00:06, 20.05it/s]


...

epoch : 18,train_positive_acc : 0.8115246098439376 train_acc : 0.3092436974789916, acc : 0.26775431632995605. positive_acc : 0.772552783109405


131it [00:06, 19.83it/s]


epoch : 19,train_positive_acc : 0.8105642256902761 train_acc : 0.3056422569027611, acc : 0.2754318416118622. positive_acc : 0.7610364683301344


83it [00:04, 19.36it/s]


KeyboardInterrupt: 

In [38]:
model_K = ViT_LRP_nan_excluded.VisionTransformer(sample_datas,split_list,seq_len=6, num_classes=9, embed_dim=36*3, depth=3,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=True, drop_rate=0.2, attn_drop_rate=0.2)
model_K = model_K.to(device)

train_net(model_K,train_loader,test_loader,n_iter=100,device=device,mode='K',lr=0.0001,optimizer_cls = optim.AdamW)


131it [00:08, 15.16it/s]


epoch : 0, train_acc : 0.25618247298919566, acc : 0.329174667596817. positive_acc : 0.699616122840691


131it [00:08, 15.54it/s]


epoch : 1, train_acc : 0.32148859543817526, acc : 0.3330134153366089. positive_acc : 0.7092130518234165


131it [00:08, 15.39it/s]


...

epoch : 26, train_acc : 0.4350540216086435, acc : 0.3214971125125885. positive_acc : 0.6976967370441459


131it [00:07, 16.40it/s]


epoch : 27, train_acc : 0.4530612244897959, acc : 0.33973127603530884. positive_acc : 0.7178502879078695


63it [00:05, 12.22it/s]


KeyboardInterrupt: 

In [6]:
model_K = ViT_LRP_nan_excluded.VisionTransformer(sample_datas,split_list,seq_len=6, num_classes=9, embed_dim=32*3, depth=5,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.2, attn_drop_rate=0.1)
model_K = model_K.to(device)

train_net(model_K,train_loader,test_loader,n_iter=100,device=device,mode='K',lr=0.0001,optimizer_cls = optim.AdamW)

131it [00:04, 29.68it/s]


epoch : 0, train_acc : 0.23169267707082833, acc : 0.3378118872642517. positive_acc : 0.690978886756238


131it [00:04, 29.62it/s]


epoch : 1, train_acc : 0.31020408163265306, acc : 0.33397310972213745. positive_acc : 0.6957773512476008


131it [00:04, 30.03it/s]


...

epoch : 41, train_acc : 0.4509003601440576, acc : 0.30422264337539673. positive_acc : 0.6900191938579654


131it [00:04, 30.05it/s]


epoch : 42, train_acc : 0.46938775510204084, acc : 0.31477925181388855. positive_acc : 0.6737044145873321


84it [00:02, 29.84it/s]


KeyboardInterrupt: 

In [ ]:
model_K = ViT_LRP_nan_excluded.VisionTransformer(sample_datas,split_list,seq_len=6, num_classes=9, embed_dim=64*3, depth=6,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.2, attn_drop_rate=0.2)
model_K = model_K.to(device)

train_net(model_K,train_loader,test_loader,n_iter=100,device=device,mode='K',lr=0.0001,optimizer_cls = optim.AdamW)

In [10]:
xx,(label_E,label_K,label_M) = next(iter(train_loader))

In [23]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(233,80),
            nn.ReLU(),
            nn.Linear(80,30),
            nn.ReLU(),
            nn.Linear(30,9)
        )
    def forward(self,x):
        x[torch.isnan(x)] = 0
        return self.mlp(x)

In [24]:
mlp = MLP()

In [25]:
mlp = mlp.to(device)

In [26]:
train_net(mlp,train_loader,test_loader,n_iter=100,device=device,mode='K',lr=0.0001)

131it [00:00, 179.46it/s]


epoch : 0,train_positive_acc : 0.4391356542617047 train_acc : 0.11692677070828332, acc : 0.17946256697177887. positive_acc : 0.5422264875239923


131it [00:00, 171.24it/s]


epoch : 1,train_positive_acc : 0.5181272509003602 train_acc : 0.17575030012004803, acc : 0.1650671809911728. positive_acc : 0.5873320537428023


131it [00:00, 175.49it/s]


...

epoch : 97,train_positive_acc : 0.8405762304921969 train_acc : 0.32869147659063624, acc : 0.2783109247684479. positive_acc : 0.761996161228407


131it [00:00, 192.29it/s]


epoch : 98,train_positive_acc : 0.8379351740696278 train_acc : 0.33085234093637456, acc : 0.2773512303829193. positive_acc : 0.7802303262955854


131it [00:00, 199.65it/s]


epoch : 99,train_positive_acc : 0.8403361344537815 train_acc : 0.33301320528211287, acc : 0.2744721472263336. positive_acc : 0.7677543186180422


In [ ]:
model_E = ViT_LRP_copy.VisionTransformer(seq_len=6, num_classes=9, embed_dim=72, depth=8,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.1, attn_drop_rate=0.1)
model_K = ViT_LRP_copy.VisionTransformer(seq_len=6, num_classes=9, embed_dim=72, depth=8,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.1, attn_drop_rate=0.1)
model_M = ViT_LRP_copy.VisionTransformer(seq_len=6, num_classes=9, embed_dim=72, depth=8,
                 num_heads=6, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0.1, attn_drop_rate=0.1)                 
for concated_data,(label_E,label_K,label_M) in train_loader:
    datas = batch_to_splited_datas(concated_data,split_list)
    emb_batch_list= batch_to_embbedings(datas,embbeding_networks) # can be used for contrastive loss
    # emb_batch_list : 임베딩 벡터들의 리스트. 얘를 이제 batch x seq x feature 행렬로 쌓음
    emb_batched_seq = torch.stack(emb_batch_list).transpose(0,1)
    attn_mask = make_attn_mask(emb_batched_seq)
    (E_score,K_score,M_score) = model_E(emb_batched_seq,attn_mask),model_K(emb_batched_seq,attn_mask),model_M(emb_batched_seq,attn_mask)
    



criterion = nn.CrossEntropyLoss()

loss_E = criterion(E_score,label_E)
loss_K = criterion(K_score,label_K)
loss_M = criterion(M_score,label_M)